In [1]:
cd ../..

e:\GitHub\computervision


In [7]:
import numpy as np
import os
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import models, transforms
from modules.ai_util import PytorchDataset, PytorchTraining

In [12]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/vit/models/01-24-2024 19-25-41/'
MODEL_LAST_EPOCH = 'epoch_8.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:\Datasets\cor-splits\sgkf-8-1-1-4000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 25
LR_EPOCH_SCHEDULE = 7
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup

In [13]:
working_folder = 'notebooks/efficientnet'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%d-%m %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [14]:
# efficientnet_b0
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(), # When converting to tensor, pytorch automatically rescales to [0,1]
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [15]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = False

# A newly defined layer is created with requires_grad=True by default
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001)

# StepLR Decays the learning rate of each parameter group by gamma every step_size epochs
# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)

dataset = PytorchDataset(DATA_DIR, data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)

model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/25
----------
train Loss: 0.6892 Acc: 0.5420
val Loss: 0.5940 Acc: 0.7780
Epoch duration: 1 m 29s

Epoch 2/25
----------
train Loss: 0.6557 Acc: 0.6432
val Loss: 0.5774 Acc: 0.7959
Epoch duration: 1 m 29s

Epoch 3/25
----------
train Loss: 0.6278 Acc: 0.6855
val Loss: 0.5668 Acc: 0.8341
Epoch duration: 1 m 28s

Epoch 4/25
----------
train Loss: 0.6074 Acc: 0.7160
val Loss: 0.5527 Acc: 0.8461
Epoch duration: 1 m 29s

Epoch 5/25
----------
train Loss: 0.5851 Acc: 0.7426
val Loss: 0.5469 Acc: 0.8413
Epoch duration: 1 m 29s

Epoch 6/25
----------
train Loss: 0.5680 Acc: 0.7598
val Loss: 0.5364 Acc: 0.8473
Epoch duration: 1 m 30s

Epoch 7/25
----------
train Loss: 0.5512 Acc: 0.7732
val Loss: 0.5268 Acc: 0.8532
Epoch duration: 1 m 29s

Epoch 8/25
----------
train Loss: 0.5436 Acc: 0.7779
val Loss: 0.5269 Acc: 0.8461
Epoch duration: 1 m 29s

Epoch 9/25
----------
train Loss: 0.5402 Acc: 0.7797
val Loss: 0.5320 Acc: 0.8401
Epoch duration: 1 m 28s

Epoch 10/25
----------
train Loss: 0.